In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
model_path = "/gdrive/MyDrive/PPNCKH/saved/TREC/DistilBERT"

# Data downloader


In [3]:
!pip install tokenizers -q
!pip install transformers -q
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 32.5 MB/s eta 0:00:00


In [4]:
# Random
from transformers import set_seed
set_seed(42)

In [5]:
from datasets import load_dataset
from datasets import DatasetDict

train_dataset = load_dataset("trec", split = "train")
test_dataset = load_dataset("trec", split = "test")

Generating train split:   0%|          | 0/5452 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset trec downloaded and prepared to /root/.cache/huggingface/datasets/trec/default/2.0.0/f2469cab1b5fceec7249fda55360dfdbd92a7a5b545e91ea0f78ad108ffac1c2. Subsequent calls will reuse this data.


Split the train into train and dev

In [6]:
train, dev = train_dataset.train_test_split(
    test_size = .2,
    seed = 42,
    stratify_by_column = "coarse_label"
).values()

In [7]:
train

Dataset({
    features: ['text', 'coarse_label', 'fine_label'],
    num_rows: 4361
})

In [8]:
dev

Dataset({
    features: ['text', 'coarse_label', 'fine_label'],
    num_rows: 1091
})

In [9]:
test_dataset

Dataset({
    features: ['text', 'coarse_label', 'fine_label'],
    num_rows: 500
})

In [10]:
dataset = DatasetDict({
    "train" : train,
    "dev" : dev,
    "test" : test_dataset
})

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'coarse_label', 'fine_label'],
        num_rows: 4361
    })
    dev: Dataset({
        features: ['text', 'coarse_label', 'fine_label'],
        num_rows: 1091
    })
    test: Dataset({
        features: ['text', 'coarse_label', 'fine_label'],
        num_rows: 500
    })
})

In [12]:
num_labels = len(dataset["train"].features["coarse_label"].names)

In [13]:
num_labels

6

# Preprocessing

In [14]:
from transformers import AutoTokenizer

In [15]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased", use_fast = True)

In [16]:
tokenizer(dataset["train"][0]["text"])

{'input_ids': [101, 10208, 1103, 2369, 1115, 1110, 16448, 1118, 170, 4354, 1761, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [17]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation = True)

In [18]:
tokenized_dataset = dataset.map(preprocess_function, batched = True, remove_columns = ["fine_label"])

Map:   0%|          | 0/4361 [00:00<?, ? examples/s]

Map:   0%|          | 0/1091 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Rename the label column to fit the concept

In [19]:
tokenized_dataset = tokenized_dataset.rename_column("coarse_label", "labels")

In [20]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 4361
    })
    dev: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 1091
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 500
    })
})

# Finetuning BERT for text classification

In [21]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

In [22]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-cased", num_labels = num_labels)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier

In [23]:
batch_size = 16
learning_rate = 2e-5
weight_decay = .01
num_train_epochs = 10

In [24]:
args = TrainingArguments(
    "/saved",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = learning_rate,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = num_train_epochs,
    weight_decay = weight_decay,
    load_best_model_at_end = True,
    metric_for_best_model = "accuracy",
    # push_to_hub = True
)

In [25]:
from datasets import load_metric
import numpy as np

In [26]:
metric = load_metric("glue", "mnli")

<ipython-input-26-1c23e2ff899a>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "mnli")


In [27]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    predictions = np.argmax(predictions, axis = 1)

    return metric.compute(predictions = predictions, references = labels)

In [28]:
trainer = Trainer(
    model,
    args,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["dev"],
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

## Training

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.293149,0.913841
2,0.446600,0.244097,0.944088
3,0.446600,0.276848,0.940422
4,0.073300,0.301381,0.943171
5,0.073300,0.331433,0.938588
6,0.023400,0.360653,0.941338
7,0.023400,0.361757,0.946838
8,0.002900,0.377346,0.942255
9,0.002900,0.380127,0.945005
10,0.002800,0.378420,0.944088


TrainOutput(global_step=2730, training_loss=0.10069286170459929, metrics={'train_runtime': 208.9281, 'train_samples_per_second': 208.732, 'train_steps_per_second': 13.067, 'total_flos': 269810835695568.0, 'train_loss': 0.10069286170459929, 'epoch': 10.0})

In [ ]:
trainer.save_model(model_path)

# Evaluation

In [29]:
import torch

In [30]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [31]:
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels = num_labels).to(device)

In [32]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

In [33]:
count_parameters(model)

+---------------------------------------------------------+------------+
|                         Modules                         | Parameters |
+---------------------------------------------------------+------------+
|       distilbert.embeddings.word_embeddings.weight      |  22268928  |
|     distilbert.embeddings.position_embeddings.weight    |   393216   |
|          distilbert.embeddings.LayerNorm.weight         |    768     |
|           distilbert.embeddings.LayerNorm.bias          |    768     |
|  distilbert.transformer.layer.0.attention.q_lin.weight  |   589824   |
|   distilbert.transformer.layer.0.attention.q_lin.bias   |    768     |
|  distilbert.transformer.layer.0.attention.k_lin.weight  |   589824   |
|   distilbert.transformer.layer.0.attention.k_lin.bias   |    768     |
|  distilbert.transformer.layer.0.attention.v_lin.weight  |   589824   |
|   distilbert.transformer.layer.0.attention.v_lin.bias   |    768     |
| distilbert.transformer.layer.0.attention.out_lin.

65786118

In [ ]:
trainer.evaluate()

{'eval_loss': 0.36175736784935,
 'eval_accuracy': 0.9468377635197067,
 'eval_runtime': 1.3886,
 'eval_samples_per_second': 785.677,
 'eval_steps_per_second': 49.69,
 'epoch': 10.0}

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_dataset["test"])

predictions = np.argmax(predictions, axis = 1)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
print(classification_report(predictions, labels))

              precision    recall  f1-score   support

           0       0.89      1.00      0.94         8
           1       0.86      0.96      0.91        84
           2       1.00      0.95      0.98       145
           3       0.98      0.97      0.98        66
           4       0.96      0.96      0.96        81
           5       0.99      0.97      0.98       116

    accuracy                           0.96       500
   macro avg       0.95      0.97      0.96       500
weighted avg       0.96      0.96      0.96       500



In [ ]:
print(f"Accuracy = {accuracy_score(predictions, labels):.6f}, \
        Precision = {precision_score(predictions, labels, average = 'macro'):.6f}, \
        Recall = {recall_score(predictions, labels, average = 'macro'):.6f}, \
        F1 = {f1_score(predictions, labels, average = 'macro'):.6f}")

Accuracy = 0.962000,         Precision = 0.948220,         Recall = 0.969031,         F1 = 0.957464


In [ ]:
print(confusion_matrix(predictions, labels))

[[  8   0   0   0   0   0]
 [  0  81   0   1   2   0]
 [  1   4 138   0   1   1]
 [  0   2   0  64   0   0]
 [  0   3   0   0  78   0]
 [  0   4   0   0   0 112]]


In [ ]:
def print_clarification(idx):
    for sent, pred, label in zip(tokenized_dataset["test"][0:idx]["text"], predictions[0:idx], labels[0:idx]):
        print(sent)
        print(f"Predicted: {pred} - Label: {label}")

In [ ]:
print_clarification(10)

How far is it from Denver to Aspen ?
Predicted: 5 - Label: 5
What county is Modesto , California in ?
Predicted: 4 - Label: 4
Who was Galileo ?
Predicted: 3 - Label: 3
What is an atom ?
Predicted: 2 - Label: 2
When did Hawaii become a state ?
Predicted: 5 - Label: 5
How tall is the Sears Building ?
Predicted: 5 - Label: 5
George Bush purchased a small interest in which baseball team ?
Predicted: 3 - Label: 3
What is Australia 's national flower ?
Predicted: 1 - Label: 1
Why does the moon turn orange ?
Predicted: 2 - Label: 2
What is autism ?
Predicted: 2 - Label: 2


In [ ]:
for i in range(len(predictions)):
    if predictions[i] != labels[i]:
        print(tokenized_dataset["test"][i]["text"])
        print(f"Predicted: {predictions[i]} - Label: {labels[i]}")

What is the speed hummingbirds fly ?
Predicted: 2 - Label: 5
The U.S. Department of Treasury first issued paper currency for the U.S. during which war ?
Predicted: 3 - Label: 1
What is the longest major league baseball-winning streak ?
Predicted: 5 - Label: 1
What is the major fault line near Kentucky ?
Predicted: 4 - Label: 1
In the late 1700 's British convicts were used to populate which colony ?
Predicted: 1 - Label: 4
What is the birthstone for June ?
Predicted: 2 - Label: 1
What is the sales tax in Minnesota ?
Predicted: 5 - Label: 1
What New York City structure is also known as the Twin Towers ?
Predicted: 1 - Label: 4
What are the spots on dominoes called ?
Predicted: 4 - Label: 1
What is the electrical output in Madrid , Spain ?
Predicted: 5 - Label: 1
What are the two houses of the Legislative branch ?
Predicted: 3 - Label: 1
What is the criterion for being legally blind ?
Predicted: 2 - Label: 1
What is foot and mouth disease ?
Predicted: 2 - Label: 1
What does the technical

Inference time measuring

In [ ]:
def predict(text, model):
    tokenized = tokenizer(text, return_tensors = "pt").to(device)
    output = model(**tokenized)[0].detach().cpu().numpy()
    
    prediction = np.argmax(output, axis = 1)[0]

    return prediction

In [ ]:
predict("How far is it from Denver to Aspen ?", model)

5

In [ ]:
from time import perf_counter

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
predicted_timelapse = []

for text in tqdm_notebook(tokenized_dataset["test"]["text"]):
    start_time = perf_counter()
    predict(text = text, model = model)
    end_time = perf_counter()

    predicted_timelapse.append(end_time - start_time)

<ipython-input-60-9fc675d8524a>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for text in tqdm_notebook(tokenized_dataset["test"]["text"]):


  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
timelapse_mean = np.mean(predicted_timelapse) * 1000
timelapse_std = np.std(predicted_timelapse) * 1000

In [ ]:
print(f"Mean: {timelapse_mean:.8f}, stdev: {timelapse_std:.8f} ms")

Mean: 9.77568926, stdev: 2.49850381 ms
